In [1]:
from audio_utils import AudioRankerSequential, wave_read_to_wav_bytes
from judge import run_speech_judge

In [2]:
import weave

# SPEECH_AUDIO_DATASET_URI = "weave:///wandb-voice-ai/voice-judge/object/generated_speech_audio:v1"
SPEECH_AUDIO_DATASET_URI = "weave:///wandb-voice-ai/voice-judge/object/generated_speech_audio_test"

weave_client = weave.init("wandb-voice-ai/voice-judge")
print("Downloading speech samples from Weave...")
ds_ref = weave.ref(SPEECH_AUDIO_DATASET_URI).get()
speech_samples = list(ds_ref.rows)

samples_to_rank = []
for i in range(3):
    samples_to_rank.append({"id": speech_samples[i]["voice_instructions_id"],
                           "audio": speech_samples[i]["audio"],
                           "instructions": speech_samples[i]["voice_instructions"],
                           })

weave: Logged in as Weights & Biases user: morgan.
weave: View Weave data at https://wandb.ai/wandb-voice-ai/voice-judge/weave


In [3]:
@weave.op
def rank_audio(sample_one, sample_two, sample_three):
    pass

_, call = rank_audio.call(samples_to_rank[0], samples_to_rank[1], samples_to_rank[2])

target_call_id = call.id

In [4]:
print(target_call_id)

01970e2d-67f4-7673-971f-265c3ec156bb


In [7]:
samples_to_rank[:3]

[{'id': 'surprised_french_veteran_20250526_2033',
  'audio': <wave.Wave_read at 0x122ad6fd0>,
  'instructions': 'Voice/Affect: raspy, weather-worn male, late-50s, clear but noticeable French accent in English.\nTone & Emotion: loud, rapid-fire surprise laced with frustrated disbelief and dark humor.\nDelivery: fast, agitated, slightly breathless; brief pause after “up there?” then punch the rest.\nPronunciation/Emphasis: stress “GTA 6,” “by now,” and “hand.” Keep French vowels on “coding.”\nLine: “What are you doing up there? (short beat) GTA 6 was supposed to be out by now — you coding the whole thing by hand or what?!”'},
 {'id': 'weary_georgia_mutter_20250526_2033',
  'audio': <wave.Wave_read at 0x122ace510>,
  'instructions': 'Voice/Affect: gravelly, life-worn Southern drawl of a late-50s Georgia man, slightly hoarse.\nTone: confused, urgent, tinged with weary frustration.\nPacing/Delivery: quick, breathy, and hushed—as if muttering under his breath but in a rush.\nEmotion: bewilde

In [5]:
ranker = AudioRankerSequential(
    [{"id": speech_samples[0]["voice_instructions_id"], 
      "audio": wave_read_to_wav_bytes(speech_samples[0]["audio"])}, 
     {"id": speech_samples[1]["voice_instructions_id"], 
      "audio": wave_read_to_wav_bytes(speech_samples[1]["audio"])},
     {"id": speech_samples[2]["voice_instructions_id"], 
      "audio": wave_read_to_wav_bytes(speech_samples[2]["audio"])}
     ],
    weave_client=weave_client,
    target_call_id=target_call_id,
)

weave: 🍩 https://wandb.ai/wandb-voice-ai/voice-judge/r/call/01970e2d-67f4-7673-971f-265c3ec156bb


weave: 📦 Published to https://wandb.ai/wandb-voice-ai/voice-judge/weave/objects/AudioSequentialRanking/versions/qN25MqRp9JQ9szXs5KkkzPJacZKdIxtYmquq8hY1Wd0


In [6]:
ranker.display_widget()

In [8]:
final_rankings = ranker.get_final_rankings()
final_rankings

[{'rank': 1,
  'id': 'hushed_georgia_rush_20250526_2033',
  'original_input_order': 2},
 {'rank': 2,
  'id': 'weary_georgia_mutter_20250526_2033',
  'original_input_order': 1},
 {'rank': 3,
  'id': 'surprised_french_veteran_20250526_2033',
  'original_input_order': 0}]

In [ ]:
for s in samples_to_rank:
    for r in final_rankings:
        if s.get("id") == r.get("id")    


audio_bytes = wave_read_to_wav_bytes(audio_obj)
audio_bytes_two = wave_read_to_wav_bytes(audio_obj_two)

rankings_with_bytes = []
for ranking in final_rankings:
    rankings_with_bytes.append({
        "id": ranking["id"],
        "audio": wave_read_to_wav_bytes(ranking["audio"]),
    })

async def example():
    # Example with file path
    result = await process_audio_with_gemini(
        system_instruction=BASE_SYSTEM_INSTRUCTION,
        prompt="Describe each voice in this audio.",
        model_name="gemini-2.0-flash",
        temperature=0.1,
        response_model=JudgeCriteria,
        audio_data=[audio_bytes, audio_bytes_two],
    )
    return result
    
# Run example
result = asyncio.run(example())
print(result.thinking)
print(result.best_voice)

run_speech_judge()